In [ ]:
# Install GDAL in Colab
!apt-get install -y gdal-bin python-gdal python3-gdal
!pip install gdal

# Import necessary libraries
from osgeo import gdal, osr
from google.colab import files
import os

# Upload the image
print("Please upload the image file (e.g., JPG).")
uploaded = files.upload()

# Get the file name of the uploaded image
input_file = list(uploaded.keys())[0]

# Define output file name
output_file = os.path.splitext(input_file)[0] + ".tif"

# Georeferencing parameters (Update as needed)
top_left_x = 50.0  # Longitude
top_left_y = 30.0  # Latitude
bottom_right_x = 100.0  # Longitude
bottom_right_y = -10.0  # Latitude

# Open the uploaded image
src_ds = gdal.Open(input_file)

# Calculate pixel size based on image dimensions
image_width = src_ds.RasterXSize
image_height = src_ds.RasterYSize
pixel_width = (bottom_right_x - top_left_x) / image_width
pixel_height = (bottom_right_y - top_left_y) / image_height

# Create a GeoTIFF file
driver = gdal.GetDriverByName('GTiff')
dst_ds = driver.Create(output_file, image_width, image_height, src_ds.RasterCount, gdal.GDT_Byte)

# Copy image data
for i in range(1, src_ds.RasterCount + 1):
    dst_ds.GetRasterBand(i).WriteArray(src_ds.GetRasterBand(i).ReadAsArray())

# Set geotransform
geotransform = (top_left_x, pixel_width, 0, top_left_y, 0, pixel_height)
dst_ds.SetGeoTransform(geotransform)

# Set spatial reference
srs = osr.SpatialReference()
srs.ImportFromEPSG(4326)  # EPSG:4326 WGS84
dst_ds.SetProjection(srs.ExportToWkt())

# Close datasets
dst_ds.FlushCache()
dst_ds = None
src_ds = None

print(f"GeoTIFF created: {output_file}")

# Allow downloading the GeoTIFF file
files.download(output_file)